In [58]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

df = pd.read_csv('datasets/test_level_a.csv')
df.rename(columns={'label': 'true_label'}, inplace=True)
sampled_df =df.sample(n=10)
sampled_df.reset_index(drop=True, inplace=True)
sampled_df

,id,tweet,true_label
0,89392,"#Conservatives VOTE VOTE VOTE, TAKE TWO FRIENDS TO POLLS to MIDTERMS in Nov. #NoComplacency‼️‼️ URL",NOT
1,96033,#Florence looks like she is getting ready to explode. Notice the eye starting to reappear. URL,NOT
2,75637,"#LisaxMichaelKors she is sooo talented, love hr too much URL",NOT
3,24957,"... Antifa comrades miss the nuance of (a lot don't, but still, people, you know): It's not that 45 is a white nationalist,"" as that's a very specific community. It's that he uses the same white supremacist rhetoric employed by WNs, which gives him dangerous company with them..""",NOT
4,95258,Aparently my computer doesn't want to operate at the moment. So I will upload the episodes Individually. Sorry for the inconvenience.,NOT
5,77649,"#Auspol Flawed though they were in many ways as individuals , Our Labor PMs have always battled on and achieved a lot for the nation. Recently, apart from gun control, what have the LNP actually done _for_ the nation?? Go on, make a list. URL",NOT
6,55832,#David #Miliband #DavidMiliband flew away in #NewYork to be with the #US #Liberals #Democrats - had to flee #Corbynists hatred of Jews,OFF
7,78266,$500 Million Funding Competition for US cities That’s nice. ⁦@USER URL,NOT
8,86382,7E91EB04 :Battle ID I need backup! Lvl 75 Celeste Omega URL,NOT
9,94816,@USER And she is a sweetheart. I hope she gets well soon. Be patient and strong Mr. Capullo.,NOT


In [59]:
df = pd.read_csv('predictions.csv')
df.drop(columns=['bert_label'], inplace=True)
df2 = pd.read_csv('predictions_svm_stems_other.csv')
df['svm_stems_other_label'] = df2['trad_label']
data = []

decriptions=["both correct", "GPT correct, SVM wrong", "GPT wrong, SvM correct", "both wrong"]
for i in range(0,4):
    for label in ["NOT", "OFF"]:
        if(i==0): filtered_df = df[(df['true_label'] == label) & (df['gpt_label'] == label) & (df['svm_stems_other_label'] == label)]
        if(i==1): filtered_df = df[(df['true_label'] == label) & (df['gpt_label'] == label) & (df['svm_stems_other_label'] != label)]
        if(i==2): filtered_df = df[(df['true_label'] == label) & (df['gpt_label'] != label) & (df['svm_stems_other_label'] == label)]
        if(i==3): filtered_df = df[(df['true_label'] == label) & (df['gpt_label'] != label) & (df['svm_stems_other_label'] != label)]
        if len(filtered_df) > 0:
            random_row = filtered_df.sample(n=1).to_dict(orient='records')[0]
            random_row['description']=decriptions[i]
            data.append(random_row)

df2 = pd.DataFrame(data)
df2

,id,tweet,true_label,gpt_label,svm_stems_other_label,description
0,10611,@USER @USER She is literally everything 👏🏼👏🏼👏🏼👏🏼👏🏼👏🏼,NOT,NOT,NOT,both correct
1,71592,#MAGA Tell me ONE good thing #trump has done as #president. I dare you! He is the worst president in history.,OFF,OFF,OFF,both correct
2,52104,"#Chequers crashes and burns in #Strasbourg along with #May's hopes of clinging to power...…..her support ebbing away in the @USER Party, in #Parliament and in the Country. It is time for her to load the Webley.....",NOT,NOT,OFF,"GPT correct, SVM wrong"
3,72037,#Texas Ted Cruz campaign is sending people solicitations disguised as official looking county summons. Elderly residents have been confused by the notice to appear. He is so skeezy. URL,OFF,OFF,NOT,"GPT correct, SVM wrong"
4,68371,#Democrats and #Liberals accuse #BrettKavanaugh and have no proof!! Meanwhile they supported #TedKennedy in his run for @USER and called him the lion of the Senate and he killed a woman. #wednesdaywisdom #news #trump URL,NOT,OFF,NOT,"GPT wrong, SvM correct"
5,78688,"#ConfirmKavanugh now, stall tactics are DC cronies only recourse. No more delays. #VoteRed to end this madness and to #MAGA",OFF,NOT,OFF,"GPT wrong, SvM correct"
6,60713,#Conservatives are going to make hay out of #Trudeau’s answer but the truth is there’s criminals everywhere and people who will commit heinous acts of violence. Whether they’re from Syria and have brownskin or are white Canadian born it’s tragic all around. URL,NOT,OFF,OFF,both wrong
7,65545,#GreatestThingsAboutThe90s liberals weren’t unhinged,OFF,NOT,NOT,both wrong


In [64]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from sklearn.preprocessing import LabelEncoder
from IPython.display import clear_output

le = LabelEncoder()
le.fit(df["true_label"])

tokenizer = AutoTokenizer.from_pretrained("openai-gpt")
if tokenizer.pad_token is None: tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model=AutoModelForSequenceClassification.from_pretrained('gpt_checkpoint/checkpoint-5958/')
pipeline=pipeline("text-classification", model=model, tokenizer=tokenizer)
clear_output()

In [65]:
text=input("Please input text: ")
result=pipeline(text)
print("text: ", text, "  label:",result[0]['label'],", confidence score:",result[0]['score'])

text:  today is raining   label: NOT , confidence score: 0.8648585081100464
